In [28]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)
from dotenv import load_dotenv
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
print(os.getcwd())

d:\data-science\DA\oil_data_exploration


In [3]:
load_dotenv();

In [4]:
base_ngl_data_name = 'ngl_'
base_vol_data_name = 'vol_'
months = 10

drilling_events = pd.read_excel(os.getenv('DRILLING_EVENT'), skiprows=2)
for month in range(1, months):
    if month == 9:
        df_name = f'{base_vol_data_name}{month}'
        globals()[df_name] = pd.read_csv(os.getenv('VOL_DATA').replace('XX', f'0{month}'), low_memory=False)
    else: 
        df_name = f'{base_ngl_data_name}{month}'
        globals()[df_name] = pd.read_csv(os.getenv('NGL_DATA').replace('XX', f'0{month}'), low_memory=False)
        df_name = f'{base_vol_data_name}{month}'
        globals()[df_name] = pd.read_csv(os.getenv('VOL_DATA').replace('XX', f'0{month}'), low_memory=False)

In [5]:
merged_ngl_df = pd.concat([ngl_6, ngl_7, ngl_8], ignore_index=True)

In [6]:
merged_vol_df = pd.concat([vol_6, vol_7, vol_8], ignore_index=True)

In [7]:
production_data = pd.read_csv(os.getenv('PRODUCTION'), low_memory=False)
minable_oil_sands_well_data = pd.read_csv(os.getenv('MINABLE_WELL_DATA'), low_memory=False)
well_infrastructure = pd.read_csv(os.getenv('WELL_INFRASTRUCTURE'), low_memory=False)

In [14]:
drilling_events.shape

(849, 7)

In [8]:
drilling_events.isnull().sum()

Well Licence Number    0
UWI with /9            0
Total Depth            0
Spud Date              0
Rig Release Date       0
Date Updated Date      0
Total # of ES          0
dtype: int64

In [15]:
merged_ngl_df.shape

(321121, 26)

In [26]:
merged_ngl_df.isnull().sum()

ReportingFacilityID       5163
ReportingFacilityName     5163
OperatorBAID              5163
OperatorName              5163
ProductionMonth              0
WellID                       0
WellLicenseNumber         5091
Field                     5091
Pool                     34055
Hours                        0
GasProduction                0
OilProduction                0
CondensateProduction         0
WaterProduction              0
ResidueGasVolume             0
Energy                       0
EthaneMixVolume              0
EthaneSpecVolume             0
PropaneMixVolume             0
PropaneSpecVolume            0
ButaneMixVolume              0
ButaneSpecVolume             0
PentaneMixVolume             0
PentaneSpecVolume            0
LiteMixVolume                0
dtype: int64

In [25]:
threshold = 0.7 * len(merged_ngl_df)
merged_ngl_df = merged_ngl_df.dropna(axis=1, thresh=threshold)

In [16]:
merged_vol_df.shape

(1617716, 30)

In [27]:
merged_vol_df.isnull().sum()

ProductionMonth                        0
OperatorBAID                           0
OperatorName                           0
ReportingFacilityID                    0
ReportingFacilityProvinceState         0
ReportingFacilityType                  0
ReportingFacilityIdentifier            0
ReportingFacilityName                  0
ReportingFacilitySubType               0
ReportingFacilitySubTypeDesc           0
ReportingFacilityLocation              0
FacilityLegalSubdivision               0
FacilitySection                        0
FacilityTownship                       0
FacilityRange                          0
FacilityMeridian                       0
SubmissionDate                        44
ActivityID                         23970
ProductID                          74653
FromToID                          276936
FromToIDProvinceState             325331
FromToIDType                      325331
FromToIDIdentifier                280889
Volume                             74653
CCICode         

In [24]:
threshold = 0.7 * len(merged_vol_df)
merged_vol_df = merged_vol_df.dropna(axis=1, thresh=threshold)

In [17]:
production_data.shape

(680, 5)

In [11]:
production_data.isnull().sum()

Date             0
Oil Monthly      0
Gas Monthly      0
Water Monthly    0
Wells Monthly    0
dtype: int64

In [18]:
minable_oil_sands_well_data.shape

(236, 14)

In [12]:
minable_oil_sands_well_data.isnull().sum()

BHID         0
CPA-ID      40
X83          0
Y83          0
Z            0
FROM         0
TO           0
LENGTH       0
CORGR        0
CALEGR       0
GRADE        0
SHALE        0
POROSITY     0
WATER        0
dtype: int64

In [19]:
well_infrastructure.shape

(658134, 72)

In [23]:
well_infrastructure.isnull().sum()

WellID                                        0
WellProvinceState                             0
WellType                                      0
WellIdentifier                                0
WellLocationException                         0
WellLegalSubdivision                          0
WellSection                                   0
WellTownship                                  0
WellRange                                     0
WellMeridian                                  0
WellEventSequence                             0
WellName                                      1
ConfidentialType                              0
ExperimentalConfidentialIndicator             0
LicenceType                                   0
LicenceNumber                                 0
LinkedFacilityProvinceState                   0
LinkedFacilityType                            0
LinkedFacilitySubType                         0
Field                                         0
FieldName                               

In [ ]:
threshold = 0.7 * len(well_infrastructure)
well_infrastructure = well_infrastructure.dropna(axis=1, thresh=threshold)